<a href="https://colab.research.google.com/github/andreianmatos/temporal_spaces_texture_gen/blob/main/styleGAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Use of an open-source PyTorch implementation of StyleGAN2 - https://github.com/lucidrains/stylegan2-pytorch - setup for training using Colab's free GPU resources and Google Drive - https://github.com/96jonesa/StyleGan2-Colab-Demo .

Training results and models are saved to the local runtime's 'results' and 'models' directories (folder icon on left bar), or to your Google Drive (give access in cell  below) in subdirectories of a parent directory named 'StyleGan2_small_set_demo'.


# StyleGan2:

<div>
<img src="https://github.com/andreianmatos/temporal_spaces_texture_gen/blob/main/results/stylegan_gif.gif?raw=true" width="400"/>
</div>

0. Login to Google (Drive)
1. Click 'Copy to Drive' above to make a runnable copy of this notebook.
2. Run this cell (click the play button in top left of cell) to connect to a runtime instance.
3. Navigate to 'Runtime > Change Runtime Type > Hardware Accelerator' and select GPU.
4. If needed, modify the variables found in the cell below to select behavior of demo.
5. Run cells.

In [ ]:
USE_DATASET = 'captures'

TRAINING_FROM_SCRATCH = True # set True if training from scratch, False if training from last checkpoint
MODEL_NAME = 'model'
MODEL_NUM_TRAIN_STEPS = 3000

LOW_NETWORK_CAPACITY = False # set True to use significantly lower network capacity
USE_GOOGLE_DRIVE_FOR_TRAINING = True # save models and results directly to your Google Drive

# 'none', 'first', 'every'
USE_ATTENTION_LAYERS = 'none' # which layers do you want attention applied to?
# dataset consists of relatively simple patterns and structures, and you don't observe significant long-range dependencies in your images

MODEL_AUGMENTATION_PROBABILITY = 0
MODEL_LEARNING_RATE = 2e-4
MODEL_IMAGE_SIZE = 64

In [ ]:
# Mounts your Google Drive so files can be saved to it. Note that this also allows
# files to be read from it, so only authorize this if you are comfortable doing so
# and/or using a disposable Google Drive account.

if USE_GOOGLE_DRIVE_FOR_TRAINING:
    from google.colab import drive
    drive.mount('/content/drive')

    !mkdir -p "/content/drive/My Drive/StyleGan2_2"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
MODEL_NETWORK_CAPACITY = 16
if LOW_NETWORK_CAPACITY:
    MODEL_NETWORK_CAPACITY = 4

MODEL_ATTENTION_LAYERS = []
if USE_ATTENTION_LAYERS == 'first':
    MODEL_ATTENTION_LAYERS = "[1]"
elif USE_ATTENTION_LAYERS == 'every':
    MODEL_ATTENTION_LAYERS = "[1,2,3,4,5,6]"

MODEL_NAME = USE_DATASET + '_' + MODEL_NAME

In [ ]:
# Installs the architecture from:
# https://github.com/lucidrains/stylegan2-pytorch

!pip install stylegan2_pytorch==0.17.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for adamp: filename=adamp-0.3.0-py3-none-any.whl size=5983 sha256=6cece76208d81ceb59f75d68508e155e0d594e795566b525ad7fb47ac79cc59f
  Stored in directory: /root/.cache/pip/wheels/c7/ad/0f/b41b1c45b18c66e5eef5d2254415af8055c7e2b0934145157d
  Created wheel for fire: filename=fire-0.5.0-py2.py3-none-any.whl size=116934 sha256=36ba8877ece2dfde8786d26d33ffa99a0b6ac5f796f408be3fd49631c3460244
  Stored in directory: /roo

In [ ]:
# Utilities for downloading publicly shared Google Drive files (from your Google Drive).

import requests

def download_file_from_google_drive(id, destination):
    URL = 'https://docs.google.com/uc?export=download'

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, 'wb') as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [ ]:
!pip install linformer

In [ ]:
# Downloads and unzips the selected dataset from your Google Drive.

import zipfile

if USE_DATASET == 'captures': #'captures'

    file_id = '1GcRwMjAZeqKGSngvNoHN0pf5AGDFtvZe'
    destination = 'captures.zip'
    download_file_from_google_drive(file_id, destination)
    zip_ref = zipfile.ZipFile('captures.zip', 'r')
    zip_ref.extractall('data/captures')
    zip_ref.close()

In [ ]:
# Chooses the appropriate subdirectory of dataset for training.
MODEL_DATA_DIR = 'data/captures/'

In [ ]:
# Establish directories for custom models.

CUSTOM_RESULTS_DIR = './results'
CUSTOM_MODELS_DIR = './models'

if USE_GOOGLE_DRIVE_FOR_TRAINING:
    CUSTOM_RESULTS_DIR = '"/content/drive/My Drive/StyleGan2_2/results"'
    CUSTOM_MODELS_DIR = '"/content/drive/My Drive/StyleGan2_2/models"'

In [ ]:
# Train custom models.

if TRAINING_FROM_SCRATCH:
    !stylegan2_pytorch --data {MODEL_DATA_DIR} --name {MODEL_NAME} --new --network_capacity {MODEL_NETWORK_CAPACITY} --batch_size 16 \
        --gradient_accumulate_every 4 --num_train_steps {MODEL_NUM_TRAIN_STEPS} --attn_layers {MODEL_ATTENTION_LAYERS} --image_size {MODEL_IMAGE_SIZE} \
        --aug_prob {MODEL_AUGMENTATION_PROBABILITY} --results_dir {CUSTOM_RESULTS_DIR} --models_dir {CUSTOM_MODELS_DIR} --learning_rate {MODEL_LEARNING_RATE}
else:
    !stylegan2_pytorch --data {MODEL_DATA_DIR} --name {MODEL_NAME} --network_capacity {MODEL_NETWORK_CAPACITY} --batch_size 16 \
        --gradient_accumulate_every 4 --num_train_steps {MODEL_NUM_TRAIN_STEPS} --attn_layers {MODEL_ATTENTION_LAYERS} --image_size {MODEL_IMAGE_SIZE} \
        --aug_prob {MODEL_AUGMENTATION_PROBABILITY} --results_dir {CUSTOM_RESULTS_DIR} --models_dir {CUSTOM_MODELS_DIR} --learning_rate {MODEL_LEARNING_RATE}

captures_model<data/captures/>:   0% 0/3000 [00:00<?, ?it/s]G: -121.57 | D: 241.11 | GP: 143208.06 | PL: 0.01 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:   1% 39/3000 [00:31<38:47,  1.27it/s]G: 4.75 | D: 2.58 | GP: 7137.02 | PL: 0.01 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:   3% 99/3000 [01:15<36:03,  1.34it/s]G: 25.61 | D: 13.02 | GP: 3012.91 | PL: 0.02 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:   5% 143/3000 [01:48<35:14,  1.35it/s]G: 9.63 | D: 2.46 | GP: 949.60 | PL: 0.02 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:   6% 188/3000 [02:21<34:30,  1.36it/s]G: 10.06 | D: 7.82 | GP: 266.40 | PL: 0.03 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:   8% 248/3000 [03:05<33:36,  1.36it/s]G: 1.73 | D: 1.20 | GP: 92.91 | PL: 0.03 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:  10% 293/3000 [03:39<33:34,  1.34it/s]G: 1.40 | D: 1.82 | GP: 17.83 | PL: 0.03 | CR: 0.00 | Q: 0.00
captures_model<data/captures/>:  11% 336/3000 [04:11<32:36,  1.36it/s]

In [ ]:
!stylegan2_pytorch --generate_interpolation --name {MODEL_NAME} --num_image_tiles 16  --image_size {MODEL_IMAGE_SIZE} --results_dir {CUSTOM_RESULTS_DIR + "/generated_interpolation"} --models_dir {CUSTOM_MODELS_DIR}

continuing from previous epoch - 2
100% 100/100 [01:00<00:00,  1.65it/s]
interpolation generated at /content/drive/My Drive/StyleGan2_2/results/generated_interpolation/captures_model/generated-02-01-2024_16-07-00


In [ ]:
number_images_to_generate = 10
for el in range(number_images_to_generate):
  !stylegan2_pytorch --generate --name {MODEL_NAME} --num_image_tiles 1  --image_size {MODEL_IMAGE_SIZE} --results_dir {CUSTOM_RESULTS_DIR  + "/generated_images"} --models_dir {CUSTOM_MODELS_DIR}

continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_16-09-08
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_16-09-15
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_16-09-20
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_16-09-26
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_16-09-32
continuing from previous epoch - 2
sample images generated at /content/drive/My Drive/StyleGan2_2/results/generated_images/captures_model/generated-02-01-2024_1

## Parameters accepted by model:

In [ ]:
# parameter                 | default   | description
#                           |           |
# data                      | ./data    | directory containing data
# results_dir               | ./results | directory for checkpoint sample images
# models_dir                | ./models  | directory for checkpoint models (saves to and loads from here)
# name                      | default   | name to identify model (all outputs will be saved to results_dir/name and models_dir/name)
# new                       | False     | if True then starts from scratch, else loads from saved checkpoint model
# load_from                 | -1        | if -1 then loads from most recent checkpoint, else loads from checkpoint number load_from
# image_size                | 128       | size of (square) images generated and for resizing of data
# network_capacity          | 16        | affects number of nodes per layer - decrease to train faster with lower output quality
# transparent               | False     | if True then uses RGBA, else uses RGB
# batch_size                | 3         | number of images per mini-batch (larger uses more GPU memory)
# gradient_accumulate_every | 5         | number of mini-batches to process before optimizing (choice depends on batch_size)
# num_train_steps           | 150000    | total steps of forward prop (counting starts from number of steps completed in loaded checkpoint)
# learning_rate             | 2e-4      | learning rate
# num_workers               | None      | if None then uses as many workers as possible from available CPU cores (for data loading)
# save_every                | 1000      | every save_every steps, a checkpoint model and sample images are saved
# generate                  | False     | if True then generates sample images from loaded model instead of training
# generate_interpolation    | False     | if True then generates .gif interpolation from loaded model instead of training, else does not
# num_image_tiles           | 8         | generated samples will be a grid of (num_image_tiles x num_image_tiles) images
# trunc_psi                 | 0.75      | affects how far generate images can be from average image (increase for more diversity) w_new = psi * w + (1 - psi) * w_avg
# fp16                      | False     | if True then uses fp16 half-precision to lower GPU memory usage (requires apex), else uses full-precision
# cl_reg                    | False     | if True then uses contrastive learning on discriminator (possibly improves stability and quality), else does not
# fq_layers                 | []        | list of layers to apply feature (intermediate representation) vector quantization to (can improve results, but not dramatically)
# fq_dict_size              | 256       | dictionary size for feature quantization
# attn_layers               | []        | list of layers to apply self-attention to while training (can be empty; do not use spaces; up to log2(image_size) - 1 layers)
# no_const                  | False     | if True then 4x4 block is learned from style vector, else styles a constant learned 4x4 block through progressive upsampling
# aug_prob                  | 0.0       | probability of applying differentiable augmentation to images fed to discriminator